# Wild Blueberry Yield Prediction

"The dataset used for predictive modeling was generated by the Wild Blueberry Pollination Simulation Model, which is an open-source, spatially-explicit computer simulation program that enables exploration of how various factors, including plant spatial arrangement, outcrossing and self-pollination, bee species compositions and weather conditions, in isolation and combination, affect pollination efficiency and yield of the wild blueberry agroecosystem. The simulation model has been validated by the field observation and experimental data collected in Maine USA and Canadian Maritimes during the last 30 years and now is a useful tool for hypothesis testing and theory development for wild blueberry pollination researches."

The aim it to predict blueberry yield

Features Unit Description:
- Clonesize m2 The average blueberry clone size in the field
- Honeybee bees/m2/min Honeybee density in the field
- Bumbles bees/m2/min Bumblebee density in the field
- Andrena bees/m2/min Andrena bee density in the field
- Osmia bees/m2/min Osmia bee density in the field
- MaxOfUpperTRange ℃ The highest record of the upper band daily air temperature during the bloom season
- MinOfUpperTRange ℃ The lowest record of the upper band daily air temperature
- AverageOfUpperTRange ℃ The average of the upper band daily air temperature
- MaxOfLowerTRange ℃ The highest record of the lower band daily air temperature
- MinOfLowerTRange ℃ The lowest record of the lower band daily air temperature
- AverageOfLowerTRange ℃ The average of the lower band daily air temperature
- RainingDays Day The total number of days during the bloom season, each of which has precipitation larger than zero
- AverageRainingDays Day The average of raining days of the entire bloom season

Resources:
- [Kaggle challenge](https://www.kaggle.com/competitions/playground-series-s3e14/overview)

In [ ]:
# importing standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'axes.titlesize': 18,
    'figure.figsize': (12, 4),
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('colorblind'), 
              rc=theme_parameters)

# Read Data

In [ ]:
df_train = pd.read_csv("./../data/blueberry_train.csv")
df_test = pd.read_csv("./../data/blueberry_test.csv")

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
# drop id column
df_train = df_train.drop(columns='id', axis=1)

# Eploratory data analysis - EDA

## Features distribution train vs validation set KDEs

In [ ]:
figure, ax = plt.subplots(4, 4, figsize=(16, 12))
ax = ax.flatten()

for index, col_name in enumerate(df_train.columns[:-1]):
    
    sns.kdeplot(data=df_train[col_name],
                label='Train',
                ax=ax[index])
    
    sns.kdeplot(data=df_test[col_name],
                label='Test',
                ax=ax[index])
   
    ax[index].set_title(col_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=3)

plt.tight_layout()

In [ ]:
figure, ax = plt.subplots(1, 2)
ax = ax.flatten()

sns.boxplot(data=df_train, 
            y='honeybee',
            ax=ax[0])

sns.boxplot(data=df_test,
            y='honeybee',
            ax=ax[1],
            color='orange')

ax[0].set_title('Train honeybee distribution')

ax[1].set_title('Test honeybee distribution')

plt.show()

In [ ]:
df_train['honeybee'].value_counts().sort_index(ascending=False).head(5)

- Outlyers of honeybees need to be dropped

In [ ]:
# dropping values >= 5 both in test and validation
df_train = df_train[df_train['honeybee'] < 5].reset_index()
df_test = df_test[df_test['honeybee'] < 5].reset_index()

In [ ]:
figure, ax = plt.subplots(1, 2)
ax = ax.flatten()

sns.boxplot(data=df_train, 
            y='honeybee',
            ax=ax[0])

sns.boxplot(data=df_test,
            y='honeybee',
            ax=ax[1],
            color='orange')

ax[0].set_title('Train honeybee distribution')

ax[1].set_title('Test honeybee distribution')

plt.show()

- The distribution od the features is consistend between train and validation sets

## Label distribution KDE

In [ ]:
sns.kdeplot(data=df_train['yield'],
            label='Train')

plt.title('Label distribution (yield)')

plt.show()

The distribution of the label yield looks similar to the distribution of the features 'fruitset', 'fruitmass' and 'seeds'. Since these features are not described (see notebook intro) we can hypotesize that they have been inferred from the label yield.

## Pearson Correlation

In [ ]:
# Generate correlation matrix
correlation_train = df_train.corr(method='pearson')

In [ ]:
# Generate a mask for the upper triangle
correlation_mask = np.triu(np.ones_like(correlation_train, dtype=bool))

In [ ]:
figure, ax = plt.subplots(figsize=(14, 10))

ax = sns.heatmap(correlation_train, 
            mask=correlation_mask, 
            cmap='mako',
            vmax=1.0, 
            vmin=-1.0, 
            center=0, 
            square=True, 
            linewidths=.5, 
            annot=True,
            annot_kws={'fontsize': 8},
            cbar_kws={"shrink":.8, 'orientation':'vertical'})


ax.set_title('Correlation heatmap')

plt.show()

- The correlation matrix shows a very similar and a high correlation behaviour between the features 'fruitset', 'fruitmass' and 'seeds and the lable 'yield', thus the above hypotesis was quite likely correct
- The feature 'clonesize' and 'honeybee' looks to be positively correlated (0.85), we can hypotesize that honeybees are attracted to blueberry cultivation with higher clonesize (maybe because the domestication of honeybees). Anyway, this doesn't appear to translate into a highet yield
- The temperature measures are duplicated, it's enough to keep just one
- Same with AverageRainingDays and RainingDays. Dropping AverageRainingDays

 ## Pairplot
 
 Plotting pairplots between main features to look at potential non-linear relaationships

In [ ]:
df_train.columns

In [ ]:
sns.pairplot(df_train[['clonesize', 'honeybee', 'bumbles', 'andrena', 'osmia', 'MaxOfUpperTRange', 'AverageRainingDays', 'fruitset', 'fruitmass', 'seeds', 'yield']],
             kind="reg",
             diag_kind='kde',
             plot_kws={'line_kws':{'color':'red'}},
             corner=True)

plt.suptitle('Features and Target Pairplots', 
             fontsize=20, 
             fontweight='bold')

plt.show()

The data are probably AI generated. Doesn't look like there are particular non-linar relationships between features.

# Preprocessing data before modeling

## Get the datasets model-ready

In [ ]:
def compute_engineered_features(data: pd.DataFrame) -> pd.DataFrame:
    
    """
    Create engineered features to have the dataset model-ready
    
    Args:
        data Pandas.DataFrame input
    
    Returns:
        data Pandas.DataFrame with engineered features
    """
    
    # Drop outlisers in 'honeybee'
    data = data[data['honeybee'] < 5]
    
    # Create a feature clonesize per honeybee
    data['clonesize per honeybee'] = data['clonesize'] * data['honeybee']
    
    return data

In [ ]:
# get the train data model-ready
df_train = compute_engineered_features(df_train.copy())

# get the validation data model-ready
df_test = compute_engineered_features(df_test.copy())

## Defining features and label

In [ ]:
features = ['clonesize', 
            'honeybee', 
            'bumbles', 
            'andrena', 
            'osmia', 
            'AverageOfUpperTRange',
            'AverageRainingDays', 
            'fruitset', 
            'fruitmass', 
            'seeds',
            'clonesize per honeybee']

label = ['yield']

## Scaling data

In [ ]:
# Saling data with StandardScaler
scaler = StandardScaler(copy=True,
                        with_mean=True,
                        with_std=True)

scaler.fit(df_train)

df_train_scaled = pd.DataFrame(scaler.transform(df_train))

# Scaling removed column names - put them back
df_train_scaled.columns = df_train.columns